In [1]:
import pandas as pd
import time
from oanda_forex_utilities import *

In [2]:
account_details = get_oanda_account_credentials("demo_account_1.2")
account_id = account_details["account_id"]
access_token = account_details["access_token"]

print(account_id)
print(access_token)

101-003-18483320-005
87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5


In [ ]:
get_extraction_time("M5")
time.sleep(1.5)
state = True
while state == True:
    df = get_candle_data(account_id, access_token,"EUR_USD", "M5", 500)
    df_with_indicators = get_macd_indicator(df)
    df_with_indicators = get_sma_indicator(df_with_indicators)
    df_with_indicators = get_rsi_indicator(df_with_indicators)
    df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]
    
    if (
            (df_with_indicators_complete["macd_signal_diff"].iloc[-1] < 0) and
            (df_with_indicators_complete["macd_signal_diff"].iloc[-2] < 0) and
            (df_with_indicators_complete["macd_signal_diff"].iloc[-3] >= 0) and
            (df_with_indicators_complete["rsi_14"].iloc[-1] >= 30) and
            (df_with_indicators_complete["price_close"].iloc[-1] < df_with_indicators_complete["sma_200"].iloc[-1])
       ):
        print(df_with_indicators_complete["time"].iloc[-1], "SELL ORDER FILLED")
        create_market_order_with_trailing_sl_only(account_id, access_token,"EUR_USD", "SELL", 5000, 10)

    elif(
            (df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0) and
            (df_with_indicators_complete["macd_signal_diff"].iloc[-2] > 0) and
            (df_with_indicators_complete["macd_signal_diff"].iloc[-3] <= 0) and
            (df_with_indicators_complete["rsi_14"].iloc[-1] <= 70 ) and
            (df_with_indicators_complete["price_close"].iloc[-1] > df_with_indicators_complete["sma_200"].iloc[-1])
        ):
        print(df_with_indicators_complete["time"].iloc[-1], "BUY ORDER FILLED")
        create_market_order_with_trailing_sl_only(account_id, access_token,"EUR_USD", "BUY", 5000, 10)
        
    else:
        print(df_with_indicators_complete["time"].iloc[-1], "NOTHING HAPPENS")
    time.sleep(300)